In [1]:
import torch
import time
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive
from networks.ensemble import Ensemble
import numpy as np
from scipy import ndimage
import random
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

def CSVLineToVec(line):
    line_list = line.strip().replace("\n", "").split(",")
    float_list = []
    for i in line_list:
        float_list.append(float(i))
    float_list = np.array(float_list)
    return float_list

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
# For single Sensor Baseline Model
TRUTH_FILE = "validation-data-baseline.txt"
OUT = "../data/oracle"
classes = [-1 for i in range(1000)]
controllers = []
sampled_dataset = SwarmDataset("../data/validation-easy-model", rank=0)

with open(os.path.join(OUT, TRUTH_FILE), "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        triplet = CSVLineToVec(line)
        classes[int(triplet[0])] = int(triplet[1])
        controllers.append(sampled_dataset[i][1])

for i in range(len(controllers)):
    print(f"{classes[i]}, {controllers[i][0]}, {controllers[i][1]}, {controllers[i][2]}, {controllers[i][3]}")

0, -0.2, -0.3, 0.2, -0.2
2, 0.3, 0.1, -0.0, 0.2
0, -0.2, -0.4, -0.3, -1.0
2, -0.8, -0.7, 0.5, 0.7
1, 0.4, 0.9, 0.2, -0.9
4, 0.9, -1.0, -0.6, -1.0
2, 1.0, 0.3, -0.2, 0.2
0, -0.7, -0.5, 0.8, -0.6
4, -0.8, 0.6, -0.3, -0.8
3, -0.3, -0.6, 0.9, 0.2
4, -0.9, 0.3, -0.8, -0.2
2, 0.4, 0.2, 0.2, 0.4
0, -0.0, -0.4, 0.6, -0.4
0, -0.1, 0.8, 0.2, 0.9
0, -0.5, -0.0, -0.0, 0.7
0, -0.1, -0.2, 0.2, 0.7
4, 0.6, -0.8, -0.7, -0.4
0, 0.2, 0.0, -0.0, -0.0
4, -0.2, 0.9, -0.3, -0.8
3, -0.7, -0.1, -0.4, 0.9
3, -0.6, -1.0, 0.7, 0.2
1, -0.5, 0.9, 0.6, 0.4
1, -0.8, 0.1, 1.0, 0.6
0, -0.9, 0.1, -0.6, 0.1
2, 0.9, -0.7, 0.5, 1.0
0, 0.3, -0.7, 0.8, 0.4
4, -0.5, 0.7, -0.8, -0.0
3, -0.9, -0.5, -0.2, 0.2
1, 0.3, 1.0, 0.8, -1.0
4, -0.9, 0.5, -0.6, -0.4
2, -0.5, 0.8, 0.6, 0.4
5, 0.5, -0.1, -0.4, -0.4
0, -0.8, 0.4, -0.9, 0.5
0, -0.0, -0.8, -0.8, 0.5
4, -0.2, 0.9, -1.0, -0.3
0, 0.5, 0.4, 0.5, -0.7
1, 0.7, 0.4, -0.7, 0.7
0, 0.3, -0.7, -0.6, 0.4
0, 0.3, -0.1, -0.2, -1.0
0, -0.3, -0.0, -0.8, 0.6
0, 0.4, -0.1, -0.5, -0.0
2, -0.2, 

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ensemble = Ensemble(size=3, output_size=5, lr=15e-4, learning_decay=0.7, decay_step=1, threshold=9.0, weight_decay=1e-4, new_model=True, init="Random")
# ensemble.load_ensemble("../checkpoints/ensembles/01-20-23-baseline", full=True)
# ensemble.load_ensemble("../checkpoints/ensembles/01-20-23-filtered-C", full=True)
# ensemble.load_ensemble("../checkpoints/ensembles/01-23-23-two-s-pre-final", full=True)
ensemble.load_ensemble("../checkpoints/ensembles/01-23-23-baseline-hil-A", full=True)
ensemble.eval_mode()

# Source the Original Dataset
sampled_dataset = SwarmDataset("../data/validation-easy-model", rank=0)
# sampled_dataset = SwarmDataset("../data/validation-two-sensor-model", rank=0)
# sampled_dataset = SwarmDataset("../data/filtered-full", rank=0)

a = []
for i in range(len(ensemble.ensemble)):
    embedded_positions = []
    for j, c in enumerate(classes):
        image, _ = sampled_dataset[j][0], sampled_dataset[j][1][0]
        image = np.expand_dims(image, axis=0)
        embed = ensemble.ensemble[i].forward(torch.tensor(image, device=device, dtype=torch.float))
        embed = embed.detach().cpu().squeeze(dim=0).numpy()
        embedded_positions.append(embed)

    # Evaluate Accuracy
    MAX_SEARCH = 30000
    correct, total = 0, 0
    for x, _classX in validation_set:
        for y, _classY in validation_set:
            if x == y:
                continue
            for z, _classZ in validation_set:
                if x == z or y == z:
                    continue
                # If _classX and _classY are both random, ignore.
                if _classX == 0 and _classY == 0:
                    continue
                if _classZ != _classX and _classX == _classY:
                    positive_dist = np.linalg.norm(embedded_positions[x] - embedded_positions[y])
                    negative_dist = np.linalg.norm(embedded_positions[x] - embedded_positions[z])
                    if positive_dist < negative_dist:
                        correct += 1
                    total += 1
                if total > 30000:
                    break
            if total > 30000:
                break
        if total > 30000:
            break

    acc = correct * 100 / total
    a.append(acc)
    print(f"Ensemble {i} ~ Accuracy: {acc}")

print(f"Average: {sum(a) / 3}")

Adjusting learning rate of group 0 to 1.5000e-03.
Adjusting learning rate of group 0 to 1.5000e-03.
Adjusting learning rate of group 0 to 1.5000e-03.
Ensemble 0 ~ Accuracy: 72.46091796940102
Ensemble 1 ~ Accuracy: 76.93076897436752
Ensemble 2 ~ Accuracy: 71.4109529682344
Average: 73.60087997066765
